In [27]:
import time
import requests
import base64
import types
import re
import warnings

class Sabre(object):
    token = None
    APIS = {
        'v1.shop.flights.fares' : ('GET', '/v1/shop/flights/fares', 'Lead Price Calendar API'),
        'v1.historical.shop.flights.fares' : ('GET', '/v1/historical/shop/flights/fares', 'Low Fare History API'),
        'v1.lists.supported.shop.flights.origins.destinations' : ('GET', '/v1/lists/supported/shop/flights/origins-destinations', 'City Pairs Lookup API'),
        'v1.shop.calendar.flights' : ('POST', '/v1.8.1/shop/calendar/flights', 'Advanced Calendar Search API'),
        'v1.lists.supported.pointofsalecountries' : ('GET', '/v1/lists/supported/pointofsalecountries', 'Point of Sale Country Code Lookup API'),
        'v1.lists.supported.shop.themes.theme' : ('GET', '/v1/lists/supported/shop/themes/{theme}', 'Theme Airport Lookup API'),
        'v1.auth.token' : ('POST', '/v1/auth/token', 'Authentication Request'),
        'v1.lists.utilities.airlines.alliances' : ('GET', '/v1/lists/utilities/airlines/alliances/', 'Airline Alliance Lookup API'),
        'v1.historical.flights.fares' : ('GET', '/v1/historical/flights/fares', 'Fare Range API'),
        'v1.forecast.flights.fares' : ('GET', '/v1/forecast/flights/fares', 'Low Fare Forecast API'),
        'v1.lists.supported.cities' : ('GET', '/v1/lists/supported/cities', 'Multi-Airport City Lookup API'),
        'v1.lists.top.destinations' : ('GET', '/v1/lists/top/destinations', 'Top Destinations API'),
        'v1.lists.supported.countries' : ('GET', '/v1/lists/supported/countries', 'Countries Lookup API'),
        'v1.lists.supported.cities.airports' : ('GET', '/v1/lists/supported/cities/{city}/airports/', 'Airports at Cities Lookup API'),
        'v1.shop.altdates.flights' : ('POST', '/v1.8.6/shop/altdates/flights', 'Alternate Date API'),
        'v1.lists.supported.shop.themes' : ('GET', '/v1/lists/supported/shop/themes/', 'Travel Theme Lookup API'),
        'v1.lists.supported.historical.seasonality.airports' : ('GET', '/v1/lists/supported/historical/seasonality/airports', 'Travel Seasonality Airports Lookup API'),
        'v2.shop.cars' : ('POST', '/v2.4.0/shop/cars', 'Car Availability (beta) API'),
        'v3.book.flights.seatmaps' : ('POST', '/v3.0.0/book/flights/seatmaps?mode=seatmaps', 'Seat Map (beta) API'),
        'v1.historical.flights.destination.seasonality' : ('GET', '/v1/historical/flights/{destination}/seasonality', 'Travel Seasonality API'),
        'v1.lists.utilities.airlines.' : ('GET', '/v1/lists/utilities/airlines/', 'Airline Lookup API'),
        'v1.lists.utilities.aircraft.equipment' : ('GET', '/v1/lists/utilities/aircraft/equipment/', 'Aircraft Equipment Lookup API'),
        'v1.shop.flights' : ('GET', '/v1/shop/flights', 'InstaFlights Search API'),
    }
    
    def __init__(self, client_id, client_secret, server='https://api.test.sabre.com'):
        self.client_id = client_id
        self.client_secret = client_secret
        self.server = server
        self.credentials = self.encode_credentials()  
        
        class Container(object):
            def __call__(self, *args, **kwargs):
                return self._call(self.endpoint, *args, **kwargs)
            
        self.api = Container()
        
        for api_name, (method, endpoint, description) in self.APIS.items():
            parts = api_name.split('.')
            obj = self.api
            for part in parts:
                if getattr(obj, part, None) is None:
                    setattr(obj, part, Container())
                obj = getattr(obj, part)
                    
            def fn(s, endpoint, *args, **kwargs):
                e = endpoint
                if '{' in endpoint:
                    e = endpoint.format(**kwargs)
                    for arg in re.findall(r'{(\w+)}', endpoint):
                        del kwargs[arg]
                
                if method == 'GET':
                    kwargs = {"params": kwargs}
                else:
                    kwargs = {"data": kwargs}
                    
                result = self.call_method(method.lower(), self.server + e, *args, **kwargs)
                assert result.status_code is 200, u"Got a {} (expecting 200): {}".format(result.status_code, result.json())
                return result.json()
            
            obj.endpoint = endpoint
            obj._call = types.MethodType(fn, obj, Container)
        
    def is_valid(self):
        self.last_check = time.time()
        if self.token and self.token['expires'] < self.last_check:
            return True
        return False
        
    def get_token(self):
        if self.is_valid():
            return self.token
        headers = {'Authorization': 'Basic ' + self.credentials}
        params = {'grant_type': 'client_credentials'}
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            r = requests.post(self.server + '/v1/auth/token', headers=headers, data=params)
        assert r.status_code is 200, 'Expecting 200 answer, got {} instead'.format(r.status_code)
        self.token = r.json()
        self.token['expires'] = self.last_check + self.token['expires_in']
        return self.token
        
    def encode_credentials(self):
        return base64.b64encode("{client_id}:{client_secret}".format(
                   client_id=base64.b64encode(self.client_id),
                   client_secret=base64.b64encode(self.client_secret)))
    
    def call_method(self, method, *args, **kwargs):
        if kwargs is None:
            kwargs = {}
        kwargs['headers'] = kwargs.get('headers', {})
        kwargs['headers'].update(Authorization='Bearer ' + self.get_token()[u'access_token'])
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            return getattr(requests, method)(*args, **kwargs)
    
    def get(self, *args, **kwargs):
        return self.call_method('get', *args, **kwargs)
    
    def post(self, *args, **kwargs):
        return self.call_method('post', *args, **kwargs)
    
    def api_list(self):
        for api_name, (method, endpoint, description) in self.APIS.items():
            print(u"{}: {} (endpoint: {} {})".format(api_name, description, method, endpoint))


In [28]:
client_id = 'V1:8y1tk3qc9kb9s0ta:DEVCENTER:EXT'
client_secret = 'hCfAs6B6'

In [29]:
sabre = Sabre(client_id, client_secret)

In [30]:
sabre.api_list()

v1.shop.flights.fares: Lead Price Calendar API (endpoint: GET /v1/shop/flights/fares)
v1.historical.shop.flights.fares: Low Fare History API (endpoint: GET /v1/historical/shop/flights/fares)
v1.lists.supported.shop.flights.origins.destinations: City Pairs Lookup API (endpoint: GET /v1/lists/supported/shop/flights/origins-destinations)
v1.lists.supported.countries: Countries Lookup API (endpoint: GET /v1/lists/supported/countries)
v1.lists.supported.pointofsalecountries: Point of Sale Country Code Lookup API (endpoint: GET /v1/lists/supported/pointofsalecountries)
v1.lists.supported.shop.themes.theme: Theme Airport Lookup API (endpoint: GET /v1/lists/supported/shop/themes/{theme})
v1.auth.token: Authentication Request (endpoint: POST /v1/auth/token)
v1.lists.utilities.airlines.alliances: Airline Alliance Lookup API (endpoint: GET /v1/lists/utilities/airlines/alliances/)
v1.historical.flights.fares: Fare Range API (endpoint: GET /v1/historical/flights/fares)
v1.forecast.flights.fares: Lo

In [31]:
sabre.api.v1.lists.supported.countries()

{u'DestinationCountries': [{u'CountryCode': u'AG',
   u'CountryName': u'Antigua And Barbuda'},
  {u'CountryCode': u'AR', u'CountryName': u'Argentina'},
  {u'CountryCode': u'AM', u'CountryName': u'Armenia'},
  {u'CountryCode': u'AW', u'CountryName': u'Aruba'},
  {u'CountryCode': u'AU', u'CountryName': u'Australia'},
  {u'CountryCode': u'AT', u'CountryName': u'Austria'},
  {u'CountryCode': u'BS', u'CountryName': u'Bahamas'},
  {u'CountryCode': u'BB', u'CountryName': u'Barbados'},
  {u'CountryCode': u'BE', u'CountryName': u'Belgium'},
  {u'CountryCode': u'BM', u'CountryName': u'Bermuda'},
  {u'CountryCode': u'BR', u'CountryName': u'Brazil'},
  {u'CountryCode': u'BG', u'CountryName': u'Bulgaria'},
  {u'CountryCode': u'CA', u'CountryName': u'Canada'},
  {u'CountryCode': u'KY', u'CountryName': u'Cayman Islands'},
  {u'CountryCode': u'CL', u'CountryName': u'Chile'},
  {u'CountryCode': u'CN', u'CountryName': u'China'},
  {u'CountryCode': u'CO', u'CountryName': u'Colombia'},
  {u'CountryCode': 

In [32]:
sabre.api.v1.lists.supported.shop.flights.origins.destinations()

{u'Links': [{u'href': u'https://api.test.sabre.com/v1/lists/supported/shop/flights/origins-destinations',
   u'rel': u'self'},
  {u'href': u'https://api.test.sabre.com/v1/lists/supported/shop/flights/origins-destinations?destinationregion=<destinationregion>&originregion=<originregion>&destinationcountry=<destinationcountry>&origincountry=<origincountry>&pointofsalecountry=<pointofsalecountry>',
   u'rel': u'linkTemplate'}],
 u'OriginDestinationLocations': [{u'DestinationLocation': {u'AirportCode': u'MCO',
    u'AirportName': u'Orlando International Airport',
    u'CityName': u'Orlando',
    u'CountryCode': u'US',
    u'CountryName': u'United States',
    u'RegionName': u'North America'},
   u'OriginDestinationLocations': u'ABE-MCO',
   u'OriginLocation': {u'AirportCode': u'ABE',
    u'AirportName': u'Lehigh Valley International Airport',
    u'CityName': u'Allentown',
    u'CountryCode': u'US',
    u'CountryName': u'United States',
    u'RegionName': u'North America'}},
  {u'Destinati

In [33]:
import spotipy

In [ ]:
spotipy.